# Binary classification

In [1]:
!wget https://zenodo.org/records/7991872/files/testing.json -O testing.json
!wget https://zenodo.org/records/7991872/files/training.json -O training.json

--2024-05-27 00:27:54--  https://zenodo.org/records/7991872/files/testing.json
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1696233 (1.6M) [text/plain]
Saving to: 'testing.json'

testing.json        100%[===================>]   1.62M  2.02MB/s    in 0.8s    

2024-05-27 00:27:55 (2.02 MB/s) - 'testing.json' saved [1696233/1696233]

--2024-05-27 00:27:56--  https://zenodo.org/records/7991872/files/training.json
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3142168 (3.0M) [text/plain]
Saving to: 'training.json'

training.json       100%[===================>]   3.00M  3.87MB/s    in 0.8s    

2024-05-27 00:27:58 (3.87 MB/s) - 'training.json' saved [314

In [2]:
!pip install -qU wandb
!pip install -q torchsampler
!pip install -qU torchmetrics

In [3]:
import os, gc, sys, yaml, json, copy
from pathlib import Path
import glob
from collections import Counter, defaultdict
from tqdm.auto import tqdm

import math
import random
import numpy as np
import pandas as pd

import cv2
import PIL
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import utils
from torchvision import transforms as T

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torchmetrics
from torchsampler import ImbalancedDatasetSampler

from IPython.display import clear_output

In [4]:
import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
CONFIG = dict(
    img_size = [512, 512],
    batch_size = 8,
    epochs = 50,
    seed = 42,
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
)

def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
#     os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
#     tf.random.set_seed(SEED)
#     keras.utils.set_random_seed(seed=SEED)
    print('seeding done!!!')

def flush():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        
seeding(CONFIG['seed'])

seeding done!!!


In [6]:
DATA_DIR = Path("/kaggle/input/aerial-dataset")
IMAGE_PATHS = glob.glob("/kaggle/input/aerial-dataset/*/*.png")

In [7]:
image_df = pd.DataFrame(IMAGE_PATHS, columns=['image_path'])

check_path = lambda path: os.path.exists(path)
get_image_dir = lambda path: int(Path(path).stem)
image_df['image_id'] = image_df['image_path'].map(get_image_dir)
image_df['exists'] = image_df['image_path'].map(check_path)
image_df['exists'].value_counts()

exists
True    10977
Name: count, dtype: int64

In [8]:
with open("/kaggle/working/training.json", "r") as f:
    data = json.loads(f.read())

In [9]:
df = pd.json_normalize(data, record_path=['images'])
df.rename(columns={'id': 'image_id'}, inplace=True)
df = df.merge(image_df, how='left', on='image_id')
df.rename(columns={'is_candidate_location': 'label'}, inplace=True)

# CONFIG = len(df['label'].unique())

In [10]:
# # Assuming 'df' is your DataFrame and 'is_candidate_location' is the column of interest
# data = df['label'].value_counts()

# # Calculate percentages
# percentages = data / data.sum() * 100

# # Define the separation of sectors
# explode = [0.1] * len(data)  # This will separate all sectors slightly. Adjust as needed.

# # Create the pie chart
# plt.figure(figsize=(10, 5))
# plt.pie(data, labels=data.index, autopct='%1.1f%%', startangle=140, explode=explode)

# # Equal aspect ratio ensures that pie is drawn as a circle.
# plt.axis('equal')  
# plt.title("Distribution of Landfill Candidate Location in Data")

# # Save the figure
# # plt.savefig("landfill_distribution_pie.png")

# # Show the plot
# plt.show()

In [11]:
class AerialDataset(utils.data.Dataset):
    
    def __init__(self, data, transform, mode='train'):
        super().__init__()
        self.data = data
        self.tsfm = transform
        self.mode = mode
        self.label = data.loc[:, 'label']
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.loc[idx, 'image_path']
        target = self.data.loc[idx, 'label']
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.tsfm(image=image)['image']
        
        return {"image": image, 
                "target": torch.tensor(target, dtype=torch.float)
               }
    
    def get_labels(self):
        return self.label

In [12]:
ts = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

ds = AerialDataset(data=df, transform=ts)
dls = utils.data.DataLoader(ds, batch_size=8, shuffle=True, num_workers=os.cpu_count())

In [13]:
# def get_transforms(height, width):
#     train_tsfm = A.Compose([
#         A.Resize(height=height, width=width),
# #         A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
# #         A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.5),
        
# #         A.OneOf([
# #             A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=5, p=1),
# #             A.RandomSnow(brightness_coeff=2.5, snow_point_lower=0.3, snow_point_upper=0.5, p=1),
# #             A.RandomFog(fog_coef_lower=0.7, fog_coef_upper=0.8, alpha_coef=0.1, p=1),
# #         ], p=0.3),
        
# #         A.OneOf([
# #             A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=1),
# #             A.RandomShadow(num_shadows_lower=1, num_shadows_upper=1, shadow_dimension=5, shadow_roi=(0, 0.5, 1, 1), p=1),
# #         ], p=0.2),
        
#         A.Normalize(mean=[0.485, 0.456, 0.406],
#                    std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ])
    
#     valid_tsfm = A.Compose([
#         A.Resize(height=height, width=width),
#         A.Normalize(mean=[0.485, 0.456, 0.406],
#                    std=[0.229, 0.224, 0.225]),
#         ToTensorV2()
#     ])
#     return {"train": train_tsfm, "eval": valid_tsfm}


def get_transforms(height, width):
    train_tsfm = A.Compose([
        # Geometric augmentations
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.5),
        # Photometric augmentations
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        A.Resize(height=height, width=width),
        # Normalization and conversion to tensor
        A.Normalize(mean=[0.485, 0.456, 0.406], 
                    std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    valid_tsfm = A.Compose([
        A.Resize(height=height, width=width),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    return {"train": train_tsfm, "eval": valid_tsfm}


def get_dataloaders(data, cfg, split="train"):
    img_size = cfg['img_size']
    height, width = img_size[0], img_size[1]
    tsfm = get_transforms(height=height, width=width)
    if split == 'train':
        tr_tsfm = tsfm['train']
        ds = AerialDataset(data=data, transform=tr_tsfm)
        labels = ds.get_labels()
        class_weights = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels))
        samples_weights = class_weights[labels]
        sampler = utils.data.WeightedRandomSampler(weights=samples_weights, 
                                                   num_samples=len(samples_weights), 
                                                   replacement=True)
#         sampler=ImbalancedDatasetSampler(ds)
        dls = utils.data.DataLoader(ds, 
                                    batch_size=cfg['batch_size'], 
                                    sampler=sampler,
                                    num_workers=os.cpu_count(), 
                                    drop_last=True, 
                                    pin_memory=True)
        
    elif split == 'valid' or split == 'test':
        eval_tsfm = tsfm['eval']
        ds = AerialDataset(data=data, transform=eval_tsfm)
        dls = utils.data.DataLoader(ds, 
                                    batch_size=2*cfg['batch_size'], 
                                    shuffle=False, 
                                    num_workers=os.cpu_count(), 
                                    drop_last=False, 
                                    pin_memory=True)
    else:
        raise Exception("Split should be 'train' or 'valid' or 'test'!!!")
    return dls

In [14]:
def check_class_distribution(data_loader):
    for i, batch in enumerate(data_loader):
        labels = batch['target']
        class_distribution = Counter(labels.numpy())
        print(f"Batch {i+1}: Class Distribution: {class_distribution}")

In [15]:
# kfold = model_selacosection.GroupKFold(n_splits=5)
kfold = model_selection.StratifiedKFold(n_splits=4, shuffle=True, random_state=2024)
x = df.index.values
y = df['label'].astype(int).values

df['fold'] = -1
for fold, (tr_idx, val_idx) in enumerate(kfold.split(x,y)):
    df.loc[val_idx, 'fold'] = fold

# Feature Pyramid Network

In [16]:
def gap2d(x, keepdims=False):
    out = torch.mean(x.view(x.size(0), x.size(1), -1), -1)
    if keepdims:
        out = out.view(out.size(0), out.size(1), 1, 1)

    return out


class ResnetFPN(nn.Module):
    def __init__(self, name, num_classes, pretrained=False, first_trainable=0):
        super(ResnetFPN, self).__init__()
        self.num_classes = num_classes
        self.first_trainable = first_trainable
        self.encoder = timm.create_model(name, pretrained=pretrained, features_only=True)
        
        # first backbone layers
        self.stage0 = nn.Sequential(self.encoder.conv1,
                                   self.encoder.bn1,
                                   self.encoder.act1,
                                   self.encoder.maxpool)
        
        # Backbone layers (bottom-up layer)
        self.stage1 = nn.Sequential(self.encoder.layer1)
        self.stage2 = nn.Sequential(self.encoder.layer2)
        self.stage3 = nn.Sequential(self.encoder.layer3)
        self.stage4 = nn.Sequential(self.encoder.layer4)
        
        if 'resnet18' in name.lower() or 'resnet34' in name.lower():
            in_chans = self.encoder.layer4[-1].conv2.out_channels
        if 'resnet50' in name.lower():
            in_chans = self.encoder.layer4[-1].conv3.out_channels
        
        out_chans = in_chans // 8
        # Top Layer
        self.toplayer = nn.Conv2d(
            in_chans, out_chans, kernel_size=1, stride=1, padding=0)

        # Lateral Layers
        self.latlayer1 = nn.Conv2d(
            in_chans // 2, out_chans, kernel_size=1, stride=1, padding=0)
        self.latlayer2 = nn.Conv2d(
            in_chans // 4, out_chans, kernel_size=1, stride=1, padding=0)
        self.latlayer3 = nn.Conv2d(
            out_chans, out_chans, kernel_size=1, stride=1, padding=0)
        
        # smooth layers
        mid_chans = in_chans // 2 - out_chans
        self.smooth1 = nn.Conv2d(in_chans // 4, out_chans, kernel_size=3, stride=1, padding=1)
        self.smooth2 = nn.Conv2d(mid_chans, out_chans, kernel_size=3, stride=1, padding=1)
        self.smooth3 = nn.Conv2d(in_chans // 2, out_chans, kernel_size=3, stride=1, padding=1)
        
        # fully connected layer
        self.fc = nn.Linear(out_chans, num_classes)
        
        # last fully connected layer
        self.classifier = nn.Linear(4*num_classes, num_classes)
        
        self.backbone = nn.ModuleList(
            [self.stage0, self.stage1, self.stage2, self.stage3, self.stage4]
        )
        
        self.newly_added = nn.ModuleList(
            [self.toplayer, self.latlayer1, self.latlayer2, self.latlayer3, self.smooth1, self.smooth2, self.smooth3,
            self.fc, self.classifier]
        )
        
    
    def forward(self, x):
        # bottom-up pathway 
        c1 = self.stage0(x)
        c2 = self.stage1(c1)
        c3 = self.stage2(c2).detach()
        c4 = self.stage3(c3)
        c5 = self.stage4(c4)
        
        # top-down pathway
        p5 = self.toplayer(c5)
        p4 = self._upsample_cat(p5, self.latlayer1(c4))
        p3 = self._upsample_cat(p4, self.latlayer2(c3))
        p2 = self._upsample_cat(p3, self.latlayer3(c2))
        
        # smoothing (de-aliasing effect)
        p4 = self.smooth1(p4)
        p3 = self.smooth2(p3)
        p2 = self.smooth3(p2)
        
        # Global Average Pooling
        p5 = gap2d(p5, keepdims=True)
        p4 = gap2d(p4, keepdims=True)
        p3 = gap2d(p3, keepdims=True)
        p2 = gap2d(p2, keepdims=True)
        
        # Flattening
        p5 = p5.view(p5.size(0), -1)
        p4 = p4.view(p4.size(0), -1)
        p3 = p3.view(p3.size(0), -1)
        p2 = p2.view(p2.size(0), -1)
        
        # Fully connected layers
        out5 = F.relu(self.fc(p5))
        out4 = F.relu(self.fc(p4))
        out3 = F.relu(self.fc(p3))
        out2 = F.relu(self.fc(p2))
        
        # concatenate the predictions (classification results) of each of the pyramid features
        out = torch.cat([out5, out4, out3, out2], dim=1)
        out = self.classifier(out)
        return out
    
    def _upsample_cat(self, x, y):
        _, _, H, W = y.size()
        upsampled_x = F.interpolate(
            x, size=(H,W), mode="nearest"
        )
        return torch.cat([upsampled_x, y], dim=1)

In [17]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

# EfficientNet

In [18]:
class AerialModel(nn.Module):
    def __init__(self, 
                 name: str, 
                 num_classes: int = 1, 
                 pretrained: bool = False, 
                 kernel_size: int = 3, 
                 stride: int = 2):
        
        super().__init__()
        self.encoder = timm.create_model(name, pretrained=pretrained, num_classes=0)
        nb_fts = self.encoder.num_features
        nb_fts = nb_fts // stride
        self.nb_fts = nb_fts if kernel_size < 3 else nb_fts - 1
        self.avg_pool = nn.AvgPool1d(kernel_size, stride=stride)
        
        self.flatten = nn.Flatten()
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.nb_fts, 768),
            nn.ReLU(),
#             nn.BatchNorm1d(768),
            nn.Dropout(0.2),
            nn.Linear(768, 64),
            nn.ReLU(),
#             nn.BatchNorm1d(256),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.flatten(x)
        x = self.avg_pool(x)
        
        outputs = self.head(x)
        return outputs

In [19]:
def shared_step(batch, criterion, params):
    image, target = batch["image"], batch["target"]
    image = image.to(params["device"], non_blocking=True)
    target = target.to(params["device"], non_blocking=True)

    logits = model.forward(image.to(torch.float32))
    loss = criterion(logits.squeeze(), target)
    
    preds = logits.sigmoid().squeeze()
    
    return {
        "loss": loss,
        "preds": preds
    }
    
    
def train(train_loader, model, criterion, optimizer, epoch, scaler, params):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    train_loss = 0
    for i, batch in enumerate(stream, start=1):
        optimizer.zero_grad(set_to_none=True)
        
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            outputs = shared_step(batch, criterion, params)
            loss =  outputs['loss']
        
#         accuracy = METRICS['accuracy'](predictions, target)
#         jaccard = METRICS['jaccard_index'](predictions, target)
#         fbeta = METRICS['fbeta_score'](predictions, target)
        
        metric_monitor.update("Loss", loss.item())
#         metric_monitor.update("Accuracy", accuracy)
#         metric_monitor.update("Jaccard", jaccard)
#         metric_monitor.update("FBeta", fbeta)
        
        train_loss += loss.detach().float()
        lr = optimizer.param_groups[0]['lr']
        _train_metrics = {
            "train/step_loss": loss.item(),
            "learning_rate": lr,
#             "train/accuracy": accuracy,
#             "train/jaccard_index": jaccard,
#             "train/fbeta_score": fbeta
        }
        
#         wandb.log({})
        if (i+1) % 50 == 0:
            wandb.log(_train_metrics)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        stream.set_description(
            "Epoch: {epoch}. Train.      {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
        )
        
    total_train_loss = train_loss / len(train_loader)
    
    flush()
    return _train_metrics, total_train_loss
        
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    valid_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(stream, start=1):
            outputs = shared_step(batch, criterion, params)
            loss =  outputs['loss']
            
#             accuracy = METRICS['accuracy'](predictions, target)
#             jaccard = METRICS['jaccard_index'](predictions, target)
#             fbeta = METRICS['fbeta_score'](predictions, target)

            metric_monitor.update("Loss", loss.item())
#             metric_monitor.update("Accuracy", accuracy)
#             metric_monitor.update("Jaccard", jaccard)
#             metric_monitor.update("FBeta", fbeta)
            
            valid_loss += loss.detach().float()
            _valid_metrics = {
                "valid/step_loss": loss.item(),
#                 "valid/loss": valid_loss,
#                 "valid/accuracy": accuracy,
#                 "valid/jaccard_index": jaccard,
#                 "valid/fbeta_score": fbeta
            }

            if (i+1) % 10 == 0:
                wandb.log(_valid_metrics)
            
            stream.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(epoch=epoch, metric_monitor=metric_monitor)
            )
            
    flush()
    total_valid_loss = valid_loss / len(val_loader)
    return _valid_metrics, total_valid_loss

In [20]:
def train_and_validate(model, train_dataset, val_dataset, params, fold=0):
    model = model.to(params['device'])
    run = wandb.init(
        project="PlasticOpticsBinaryClassification",
        resume="allow"
    )
    
    artifact = wandb.Artifact(f"aerialBiFPNModel_fold_{fold}", type="model")
    train_loader = get_dataloaders(train_data, cfg=CONFIG, split='train')
    val_loader = get_dataloaders(valid_data, cfg=CONFIG, split="valid")
    
    criterion = nn.BCEWithLogitsLoss().to(params["device"])
    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
    scaler = torch.cuda.amp.GradScaler()
    
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 
#                                                                      params['epochs'], 
#                                                                      eta_min=0)
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=params['epochs'], eta_min=0)
    lr_reduce = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=4, verbose=True)
    
    best_metric = np.inf
    loss_min = np.inf
    es = 0
    ES_RATIO = 0.25 if params['epochs'] < 10 else 0.20
#     seg_weights_file = "aerialwaste_binary_fpn_weights.pth"
    weights_file = "aerialwaste_binary_classification_fold_{fold}_epoch_{epoch}.pth"
    for epoch in range(1, params["epochs"] + 1):
        scheduler.step()
        _train_metrics, train_loss = train(train_loader, model, criterion, optimizer, epoch, scaler, params)
        _valid_metrics, val_loss = validate(val_loader, model, criterion, epoch, params)
        
        _train_metrics["train/loss"] = train_loss
        _valid_metrics["valid/loss"] = val_loss
        lr_reduce.step(val_loss)
        wandb.log({**_train_metrics, **_valid_metrics})
        if val_loss < best_metric:
            print(f"Best metric: ({best_metric:.6f} --> {val_loss:.6f}). Saving model ...")
#             torch.save(model.module.state_dict(), f"{name}_fold_{fold}.pth")
            weights_file.format(fold=fold, epoch=epoch)
            torch.save(model.state_dict(), weights_file)
            best_metric = val_loss
            es = 0
            if epoch == 1:
                artifact.add_file(weights_file)
                run.log_artifact(artifact)
            else:
                draft_artifact = wandb.Artifact(f"aerialMultiFPNModel_fold_{fold}", type="model")
                draft_artifact.add_file(weights_file)
                run.log_artifact(draft_artifact)
                
        else:
            es += 1
            
        if es > math.ceil(ES_RATIO*params['epochs']):
            print(f"Early stopping on epoch {epoch} ...")
            break
            
    wandb.config = params
    wandb.finish()
    flush()

In [ ]:
CONFIG['lr'] = 5e-3

for fold in range(4):
#     model = ResnetFPN(name='resnet34', num_classes=1, pretrained=True)
    model = AerialModel(name='efficientnet_b0.ra_in1k', pretrained=True)
    train_data = df[df['fold'] != fold].reset_index(drop=True)
    valid_data = df[df['fold'] == fold].reset_index(drop=True)
    train_and_validate(model, train_data, valid_data, params=CONFIG, fold=fold)
    
gc.collect()
flush()

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

wandb: Currently logged in as: samu2505. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

...

  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

learning_rate,██████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
train/loss,█▆▅▅▄▄▃▃▃▂▂▁▁▁▁
train/step_loss,▅▄▄▃▅▂█▅▅▃▄▄▄▄▃▃▂▁▂▃▁▅▁▃▂▁▄▃▃▂▂▁▂▁▂▂▂▁▂▁
valid/loss,█▆▅▄▆▄▄▃▂▄▂▁▁▁▁
valid/step_loss,▅▅▅▄▅█▂▃▆▄▂▄▂▇▃▃▄▅▇▅▂▅▂▁▂▅▇▃▂▄▃▁▃▁▅▃▁▂▃▅
learning_rate,0.0
train/loss,0.3541
train/step_loss,0.15652
valid/loss,0.31374
valid/step_loss,0.45446


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/784 [00:00<?, ?it/s]

In [ ]:
# batch = next(iter(dls))
# encoder = timm.create_model('tf_efficientnet_b7.ra_in1k', pretrained=False, features_only=True)
# encoder = timm.create_model('resnet18', pretrained=False, features_only=True)
# encoder = timm.create_model('resnet18', pretrained=False, features_only=True)
# model = ResnetFPN(name='resnet34', num_classes=1)
# model.eval()
# out = model(batch['image'])
# out = encoder(batch['image'])

In [ ]:
# class FPNModel(pl.LightningModule):
#     def __init__(self, name, num_classes, pretrained=False, lr=1e-3):
#         super().__init__()
#         self.save_hyperparameters()
#         self.model = ResnetFPN(name=name, num_classes=num_classes, pretrained=pretrained)
#         self.loss_fn = nn.BCEWithLogitsLoss()
#     def forward(self, x):
#         return self.model(x)
    
#     def training_step(self, batch, batch_idx):
#         imgs, labels = batch['image'], batch['target']
#         preds = self.forward(imgs)
#         loss = self.loss_fn(preds.squeeze(), labels)
#         self.log('loss', loss, on_step=True, on_epoch=True, prog_bar=True)
#         return loss
    
#     def configure_optimizers(self):
#         max_epochs = self.trainer.max_epochs
#         optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
# #         scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# #         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs, eta_min=0)
        
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#             optimizer, T_0=10, T_mult=2)
        
# #         scheduler = torch.optim.lr_scheduler.OneCycleLR(
# #             optimizer=optimizer, epochs=max_epochs,
# #             pct_start=0.0, steps_per_epoch=self.steps_per_epoch,
# #             max_lr=self.hparams.lr, div_factor=25, final_div_factor=4.0e-01
# #         )
        
#         return [optimizer], [scheduler]

In [ ]:
# net = FPNModel(name='resnet18', num_classes=1)
# trainer = pl.Trainer(max_epochs=10)
# trainer.fit(net, train_dataloaders=dls)

In [ ]:
# torch.save(net.state_dict(), 'aerial_binary.pth')

## Cross entropy loss

In [ ]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.reduction = reduction

#     def forward(self, inputs, targets):
#         BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
#         pt = torch.exp(-BCE_loss)  # prevents nans when probability 0
#         F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

#         if self.reduction == 'mean':
#             return torch.mean(F_loss)
#         elif self.reduction == 'sum':
#             return torch.sum(F_loss)
#         else:
#             return F_loss

In [ ]:
# y = df['label'].astype(int).values
# # neg_examples, pos_examples = np.bincount(df['label'].astype(int).values)
# # pos_weights = torch.tensor([neg_examples / pos_examples])
# # pos_weights
# pos_weights = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y))
# # np.unique(df['label'].astype(int).values, return_counts=True)

In [ ]:
# class AerialLightningModel(pl.LightningModule):
#     def __init__(self, 
#                  name: str, 
#                  num_classes: int = 0, 
#                  pretrained: bool = False, 
#                  kernel_size: int = 3, 
#                  stride: int = 2, 
#                  lr: float = 3e-4):
        
#         super().__init__()
#         self.save_hyperparameters()
#         self.encoder = timm.create_model(name, pretrained=pretrained, num_classes=0)
#         nb_fts = self.encoder.num_features
#         nb_fts = nb_fts // stride
#         self.nb_fts = nb_fts if kernel_size < 3 else nb_fts - 1
#         self.avg_pool = nn.AvgPool1d(kernel_size, stride=stride)
        
#         self.flatten = nn.Flatten()
#         self.head = nn.Sequential(
#             nn.Dropout(0.2),
#             nn.Linear(self.nb_fts, 768),
#             nn.ReLU(),
# #             nn.BatchNorm1d(768),
#             nn.Dropout(0.2),
#             nn.Linear(768, 256),
#             nn.ReLU(),
# #             nn.BatchNorm1d(256),
#             nn.Dropout(0.2),
#             nn.Linear(256, num_classes)
#         )
        
#         self.loss_fn = nn.BCEWithLogitsLoss()
#         self.accuracy = torchmetrics.Accuracy(task='binary')
#         self.recall = torchmetrics.Recall(task='binary')
#         self.precision = torchmetrics.Precision(task='binary')
#         self.step_outputs = []
        
#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.flatten(x)
#         x = self.avg_pool(x)
        
#         outputs = self.head(x)
#         return outputs
    
#     def freeze_encoder(self, flag):
#         for param in self.encoder.parameters():
#             param.requires_grad = not flag
    
#     def shared_step(self, batch, stage):
#         imgs, labels = batch['image'], batch['target']
#         preds = self.forward(imgs)
#         loss = self.loss_fn(preds.squeeze(), labels)
# #         preds = (preds.sigmoid().squeeze() > 0.5).float()
#         preds = preds.sigmoid().squeeze()
#         acc = self.accuracy(preds, labels)
#         recall = self.recall(preds, labels)
#         precision = self.precision(preds, labels)
        
#         self.log(f'{stage}_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
#         self.log(f'{stage}_acc', acc, on_step=False, on_epoch=True, prog_bar=False)
#         self.log(f'{stage}_recall', recall, on_step=False, on_epoch=True, prog_bar=False)
#         self.log(f"{stage}_precision", precision, on_step=False, on_epoch=True, prog_bar=False)
        
#         output = {
#             f"{stage}_loss": loss,
#             f"{stage}_acc": acc,
#             f"{stage}_recall": recall,
#             f"{stage}_precision": precision,
#             f"{stage}_labels": labels,
#             f"{stage}_preds": preds
#         }
#         self.step_outputs.append(output)
#         return output
    
#     def training_step(self, batch, batch_idx):
#         output = self.shared_step(batch, 'train')
#         loss = output['train_loss']
#         return loss
    
#     def validation_step(self, batch, batch_idx):
# #         output = self.shared_step(batch, 'val')
# #         labels, preds = output['val_labels'], output['val_preds']
# #         if batch_idx % 10 == 1:
# #             sys.stdout.write('\033[F'*n)
# #             sys.stdout.write('\033[K')
# #             print(f"Batch idx: {batch_idx} -> Labels: {labels}, Preds: {preds}")
# #             sys.stdout.flush()
            
# #         clear_output(wait=True)
#         return self.shared_step(batch, 'val')
            

#     def test_step(self, batch, batch_idx):
#         return self.shared_step(batch, 'test')

#     def predict_step(self, batch, batch_idx, dataloader_idx=0):
#         return self(batch)
    
#     def configure_optimizers(self):
#         max_epochs = self.trainer.max_epochs
#         optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
# #         scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# #         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs, eta_min=0)
        
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#             optimizer, T_0=10, T_mult=2)
        
# #         scheduler = torch.optim.lr_scheduler.OneCycleLR(
# #             optimizer=optimizer, epochs=max_epochs,
# #             pct_start=0.0, steps_per_epoch=self.steps_per_epoch,
# #             max_lr=self.hparams.lr, div_factor=25, final_div_factor=4.0e-01
# #         )
        
#         return [optimizer], [scheduler]

In [ ]:
# CONFIG['patience'] = 8 if CONFIG['epochs'] < 50 else 12
# for fold in range(4):
#     train_data = df[df['fold'] != fold].reset_index(drop=True)
#     valid_data = df[df['fold'] == fold].reset_index(drop=True)
#     train_dls = get_dataloaders(train_data, cfg=CONFIG, split='train')
#     valid_dls = get_dataloaders(valid_data, cfg=CONFIG, split="valid")

#     net = AerialLightningModel(name='efficientnet_b0.ra_in1k', 
#                                num_classes=1, 
#                                pretrained=True, 
#                                lr=1e-5)
#     # net.freeze_encoder(True)

#     wandb_logger = WandbLogger(project="Plastic-Optics-classification",
#                                checkpoint_name=f"aerialBinary_fold_{fold}", 
#                                log_model="all")

#     callbacks = [
#         ModelCheckpoint(save_weights_only=True, 
#                         mode="min", 
#                         monitor="val_loss"),  # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
#         LearningRateMonitor("epoch"),
#         EarlyStopping(monitor="val_loss", min_delta=0.0, patience=CONFIG['patience'], verbose=False, mode="min"),
#     ]

#     trainer = pl.Trainer(max_epochs=CONFIG['epochs'], logger=wandb_logger, callbacks=callbacks)
# #     trainer = pl.Trainer(max_epochs=10)
#     trainer.fit(net, train_dataloaders=train_dls, val_dataloaders=valid_dls)
#     break
# wandb.finish()

In [ ]:
# help(torchmetrics.Precision(task='binary', num_classes=2))
# help(torchmetrics.Recall(task='binary'))